In [51]:
from time import sleep, time, ctime
from threading import Thread, Lock, current_thread, BoundedSemaphore
import atexit
from random import randrange

In [2]:
def count(n):
    while n > 0:
        n -= 1
#         if n % 10000000 == 0:
#             print(n)

In [3]:
%%time
n = 10000000*5
count(n)

CPU times: user 2.56 s, sys: 4 ms, total: 2.56 s
Wall time: 2.56 s


Try threads

In [4]:
%%time
t0 = time()
t1 = Thread(target = count, args = (n/2,))
t1.start()
t2 = Thread(target = count, args = (n/2,))
t2.start()
t1.join()
t2.join()

CPU times: user 2.67 s, sys: 0 ns, total: 2.67 s
Wall time: 2.66 s


In [5]:
class Func(object):
    
    def __init__(self, func, args, **kwargs):
        self.func = func
        self.args = args
        self.kwargs = kwargs
        
    def __call__(self):
        return self.func(*self.args, **self.kwargs)

In [6]:
%%time
f = Func(count, (n/2,))
t1 = Thread(target = f)
t2 = Thread(target = f)
t1.start()
t2.start()
t1.join()
t2.join()

CPU times: user 2.68 s, sys: 8 ms, total: 2.69 s
Wall time: 2.69 s


In [7]:
class MyThread(Thread):
    
    def __init__(self, name, func, args):
        super().__init__()
        self.func = func
        self.args = args
        self.name = name
        
    def run(self):
        print('Thread `{name}` starting at: {t}'.format(name=self.name,t=ctime()))
        self.rez = self.func(*self.args)
        print('Thread `{name}` finished at: {t}'.format(name=self.name,t=ctime()))

In [8]:
%%time
t1 = MyThread('t1', count, (n/2,))
t2 = MyThread('t2', count, (n/2,))
t1.start()
t2.start()
print(current_thread().name)
t1.join()
t2.join()

Thread `t1` starting at: Tue Aug  8 12:26:05 2017
MainThread
Thread `t2` starting at: Tue Aug  8 12:26:05 2017
Thread `t1` finished at: Tue Aug  8 12:26:07 2017Thread `t2` finished at: Tue Aug  8 12:26:07 2017

CPU times: user 2.77 s, sys: 12 ms, total: 2.78 s
Wall time: 2.77 s


Using Sync Primitives

In [52]:
class CleanOutputSet(set):
    def __str__(self):
        return ', '.join(x for x in self)

lock = Lock()
loops = (randrange(2,5) for x in range(randrange(3,7)))
remaining = CleanOutputSet()

def loop(nsec):
    myname = current_thread().name
    with lock:
        remaining.add(myname)
        print('[{0}] Started {1}'.format(ctime(), myname))
    sleep(nsec)
    with lock:
        remaining.remove(myname)
        print('[{0}] Completed {1} ({2} secs)'.format(ctime(), myname, nsec))
        print('   (remaining: {0})'.format(remaining or 'NONE'))

def main():
    for pause in loops:
        Thread(target=loop, args=(pause,)).start()
    
@atexit.register
def _atexit():
    print('all DONE at:', ctime())

main()

[Tue Aug  8 13:52:45 2017] Started Thread-92
[Tue Aug  8 13:52:45 2017] Started Thread-93
[Tue Aug  8 13:52:45 2017] Started Thread-94
[Tue Aug  8 13:52:45 2017] Started Thread-95
[Tue Aug  8 13:52:47 2017] Completed Thread-92 (2 secs)
   (remaining: Thread-93, Thread-94, Thread-95)[Tue Aug  8 13:52:47 2017] Completed Thread-93 (2 secs)

   (remaining: Thread-94, Thread-95)
[Tue Aug  8 13:52:47 2017] Completed Thread-94 (2 secs)
   (remaining: Thread-95)
[Tue Aug  8 13:52:48 2017] Completed Thread-95 (3 secs)
   (remaining: NONE)


Using Semaphore

In [190]:
lock = Lock()
MAX = 10
service = BoundedSemaphore(MAX)

def add_candy():
    with lock:
        print('Refilling candy...', end = '')
        try:
            service.release()
#         except Exception as error:
#             print('Error occured: {0}'.format(str(type(error).__name__ + ' : ' + ' '.join(error.args))))
        except ValueError:
            print('Sorry, it is full')
        else:
            print('OK')
        print('[{0}]'.format(service._value))

def buy_candy():
    with lock:
        print('Buying candy...', end='')
        if service.acquire(False):
            print('OK')
        else:
            print('Sorry, it is empty')
        print('[{0}]'.format(service._value))
            
def producer(loops):
    for i in range(loops):
        add_candy()
        sleep(randrange(2))

def consumer(loops):
    for i in range(loops):
        buy_candy()
        sleep(randrange(2))   
        
def main():
    nloops = randrange(3,5)
    nloops_buy = randrange(nloops, nloops + MAX + 4)
    print('Buy: {0}, Sell: {1}\n'.format(nloops_buy, nloops))
    c = Thread(target=consumer, args = (nloops_buy,))
    p = Thread(target=producer, args = (nloops,))
    c.start()
    p.start()
    
main()

Buy: 16, Sell: 3
Buying candy...
OK
[9]
Buying candy...OK
[8]
Buying candy...OK
[7]
Buying candy...OK
[6]
Refilling candy...OK
[7]
Buying candy...OK
[6]
Refilling candy...OK
[7]
Buying candy...OK
[6]
Buying candy...OK
[5]
Refilling candy...OK
[6]
Buying candy...OK
[5]
Buying candy...OK
[4]
Buying candy...OK
[3]
Buying candy...OK
[2]
Buying candy...OK
[1]
Buying candy...OK
[0]
Buying candy...Sorry, it is empty
[0]
Buying candy...Sorry, it is empty
[0]
Buying candy...Sorry, it is empty
[0]


SFTP

In [147]:
import pysftp

In [169]:
host = 'ftp.kroll.com'
user = 'emoreno'
passwd = '6LO7330Q'

In [170]:
cnopts = pysftp.CnOpts()
cnopts.hostkeys = None
sftp = pysftp.Connection(host, username=user, password=passwd, cnopts=cnopts)

In [171]:
sftp.listdir()
sftp.pwd

['KrollPilotData.zip', 'System Volume Information']

'/'

In [ ]:
rt_file = 'Collected Reports.zip'
local_path = os.path.join('/home/karimlulu/Downloads', rt_file)
sftp.get(rt_file, localpath=local_path)

In [172]:
sftp.close()

Queue

In [177]:
from queue import Queue

In [209]:
def consume(q):
    while True:
        name = current_thread().name
        print('[{0}] Thread {1} started to get, size = {2}'.format(ctime(), name, q.qsize()))
        item = q.get()
        sleep(randrange(3))
        print('[{0}] Thread {1} consumed, size = {2}'.format(ctime(), name, q.qsize()))
        q.task_done()
  

def produce(q, loops):
    for k in range(loops):
        name = current_thread().name
        print('[{0}] Thread {1} started to put, size = {2}'.format(ctime(), name, q.qsize()))
        item = k
        q.put(k)
        sleep(randrange(1))
        print('[{0}] Thread {1} put, size = {2}'.format(ctime(), name, q.qsize()))

        
def main():
    loops = randrange(3,5)
    q = Queue(maxsize=10)
    c = Thread(name='ConsumerThread', target=consume, args=(q,))#, daemon = True)
    p = Thread(name='ProducerThread', target=produce, args=(q, loops))#, daemon = True)
    c.start()
    p.start()
    
    q.join()
    
main()

[Tue Aug  8 19:40:04 2017] Thread ProducerThread started to put, size = 0[Tue Aug  8 19:40:04 2017] Thread ConsumerThread started to get, size = 0
[Tue Aug  8 19:40:04 2017] Thread ProducerThread put, size = 1

[Tue Aug  8 19:40:04 2017] Thread ProducerThread started to put, size = 1
[Tue Aug  8 19:40:04 2017] Thread ProducerThread put, size = 2
[Tue Aug  8 19:40:04 2017] Thread ProducerThread started to put, size = 2
[Tue Aug  8 19:40:04 2017] Thread ProducerThread put, size = 2
[Tue Aug  8 19:40:06 2017] Thread ConsumerThread consumed, size = 2
[Tue Aug  8 19:40:06 2017] Thread ConsumerThread started to get, size = 2
[Tue Aug  8 19:40:06 2017] Thread ConsumerThread consumed, size = 1
[Tue Aug  8 19:40:06 2017] Thread ConsumerThread started to get, size = 1
[Tue Aug  8 19:40:08 2017] Thread ConsumerThread consumed, size = 0
[Tue Aug  8 19:40:08 2017] Thread ConsumerThread started to get, size = 0
